In [35]:
import re
import os
import nltk
import pickle
import itertools
import pandas as pd

# file prep
1. open pickled dataframe as `fics_df`  
2. open dictionary of english language (source: [english dictionary](https://github.com/dwyl/english-words/blob/master/words.zip))  
3. split dictionary into list as `en_dict`

In [16]:
fics_df_sl = pickle.load(open('fics_df_scraped_lang.pkl', 'rb'))
fics_df_m = pickle.load(open('fics_df_manual.pkl', 'rb'))
en_dict_txt = open('../words/words.txt').read()
en_dict = en_dict_txt.split("\n")

In [17]:
fics_df_m.describe()

,L1,author,title,tags,summary,notes,endnotes,work,rating,warnings,category,fandom,ships,charas,freeform
count,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
unique,6,13,97,1,1,1,1,97,1,1,1,1,1,1,1
top,Polish,thunderstorms_and_snow,The Night Heir,[],NaN,NaN,NaN,['\n\nChapter 1\n\n\n\nTHE COLD WATER of the l...,[],[],[],[],[],[],[]
freq,31,23,1,97,97,97,97,1,97,97,97,97,97,97,97


4. create list of all tags used to tag works  
5. flatten list  
6. create flattened token list from flattened list

In [18]:
fics_df = pd.concat([fics_df_sl, fics_df_m], ignore_index=True)

In [19]:
list_of_lists = [[tag for tag in ser] for ser in fics_df['tags']]
tags = list(itertools.chain.from_iterable(list_of_lists))
tags_toks = set(list(itertools.chain.from_iterable([nltk.word_tokenize(tag) for tag in tags])))

7. create list of lowercase tag tokens  
8. clear up most non-words  

In [20]:
all_words = [x.lower() for x in tags_toks]
all_words = set(nltk.word_tokenize(re.sub(r'[,.!@#$%^&*()_\-+=\{}\[\]:;\'?/\\\`~"™0-9|]', ' ', str(all_words))))

In [21]:
pickle.dump(all_words, open('word_list.pkl', 'wb'))

# defining function: word_acc(list)
1. collect count of set of words in English dictionary OR in tags  
    this ensures that character names, etc. are counted as words. not 100% fool-proof, but corrects much of the data  
2. collect count of words NOT counted above  
    this is for debugging purposes (checking that all words are accounted for, etc.)  
    unfortunately this includes conjunctions---I plan on coming back to fix this in the future
3. return ratio of words used that are actual words and those that are not  


In [22]:
def word_acc(list):
    i = 0
    j = 0
    for w in set(list):
        if w.lower() in en_dict:
            i = i + 1
        else:
            if w.lower() in all_words:
                i = i + 1
            else:
                j = j + 1
    return i/len(set(list))

# df updates
adding the is-the-word-actually-a-word accuracy ratio (`word_acc(list)`)  
tokens  
word count  
sentence count  
average length of words  
K-band data  
n-grams  
freqdist  
condfreqdist  

In [26]:
import stanza
nlp = stanza.Pipeline('en')

2025-04-04 00:17:28 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-04 00:17:29 INFO: Downloaded file to C:\Users\jbean\stanza_resources\resources.json
2025-04-04 00:17:31 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2025-04-04 00:17:31 INFO: Using device: cpu
2025-04-04 00:17:31 INFO: Loading: tokenize
2025-04-04 00:17:34 INFO: Loading: mwt
2025-04-04 00:17:34 INFO: Loading: pos
2025-04-04 00:17:36 INFO: Loading: lemma
2025-04-04 00:17:37 INFO: Loading: constituency
2025-04-04 00:17:38 INFO: Loading: depparse
2025-04-04 00:17:38 INFO: Loading: sentiment
2025-04-04 00:17:39 INFO: Loading: ner

In [31]:
nlp(fics_df['work'][0]).sentences[18].text

'Why?” “It’s September 30th?” “So?'

STANFORD. THAT IS NOT ONE SENTENCE IT IS THREE. disappointed :(
    

In [33]:
def tokenize(doc):
    toks = []
    for sentence in nlp(x):
        for token in sentence:
            toks.append(token)
    return toks

In [36]:
fics_df['toks'] = fics_df.work.map(lambda x : tokenize(x))

NameError: name 'x' is not defined